La figura muestra un esquema de un sistema de dos estanques de agua conectados por dos cañerías con una bifurcación y una unión. 

Determine el caudal de estanque a estanque, cuando la válvula en la sección 2 está completamente cerrada.


![Ejemplo 3](M1_E3.png)

Datos:
* Cañerías de acero comercial
* L2 = 90 m
* L3 = 150 m
* 2 codos de 45º
* válvulas de bola completamente abiertas
* 1 codo de 90° en cada tramo (2 y 3)
* Uniones en T son despreciadas para el cálculo de pérdidas de carga.




In [1]:
import CoolProp.CoolProp as cp
import fluids as fld
import numpy as np
from scipy import constants
from fluids.units import *
from scipy import optimize as op

In [6]:
D_1 = 4 * constants.inch * u.m #Pulgadas a metros
D_2 = 3 * constants.inch * u.m  #Pulgadas a metros
D_3 = 4 * constants.inch * u.m  #Pulgadas a metros
D_4 = 4 * constants.inch * u.m  #Pulgadas a metros

L_1 = 3 * u.m   #metros
L_2 = 90 * u.m  #metros
L_3 = 150 * u.m #metros
L_4 = 3 * u.m 

z = 1 * u.m     #metros

material = 'Steel'
epsilon = fld.material_roughness(material)*u.m

fluid = "water"
T_o = 300 *u.K #Kelvin
rho = cp.PropsSI('D','T', T_o.magnitude, 'P',101325,fluid)*u.kg/u.m**3
mu = cp.PropsSI('V','T', T_o.magnitude, 'P',101325,fluid)*u.Pa*u.s

Si la válvula en la cañería 2 está cerrada, no hay flujo en 2

$v_2 = 0\\$
$\dot{V}_2 = 0$

La ecuación de Bernouilli se puede hacer por cada uno de los tramos: 1, 2, 3 y 4. Luego pueden sumarse entre sí. 
Haciendo Bernoulli por 1, 3 y 4, queda:

 $\rho g z_{AB}=\left( {\frac{1}{2}\rho V_{1}^{2}\left( {\frac{f_1 L_1}{D_1}+K_1} \right)} \right)+\frac{1}{2}\rho V_{3}^{2}\left( {\frac{f_3 L_3}{D_3}+K_3} \right) + \left( {\frac{1}{2}\rho V_{4}^{2}\left( {\frac{f_4 L_4}{D_4}+K_4} \right)} \right)$

 Como no hay flujo en 2 y como $D_1=D_3=D_4$, entonces $V_1 =V_3 =V_4$.
 Luego

  $\rho g z_{AB}= \frac{1}{2}\rho V_{1}^{2}\left(f_1 \frac{(L_1 + L_3 + L_4) }{D_1}+ \sum_i{K_i} \right)$

In [7]:
g = constants.g * u.m / u.s**2

K_elbow_90 = fld.fittings.bend_rounded_Crane(Di=D_1.magnitude, bend_diameters=1, angle=90)*u.dimensionless
K_globe = fld.fittings.K_gate_valve_Crane(D_1.magnitude, D_1.magnitude, 0)*u.dimensionless
K_exit = fld.exit_normal()*u.dimensionless
K_entrance = fld.entrance_sharp()*u.dimensionless

DP_g = rho * g * z

In [9]:
def flow_134_o(V_dot_aux):
    V_dot = V_dot_aux * u.m**3/u.s
    K_a = fld.K_branch_diverging_Crane(D_1, D_1, 0*u.m**3/u.s, V_dot, angle=90*u.degree)
    K_b = fld.K_run_converging_Crane(D_1, D_2,V_dot, 0*u.m**3/u.s, angle=90*u.degree)
    DP_f = fld.one_phase_dP(V_dot*rho, rho, mu, D_1 ,epsilon, L_1 + L_3 + L_4)
    DP_s = (K_elbow_90 + K_globe +K_exit + K_entrance + K_a + K_b ) * ((V_dot*4)/(np.pi*D_1**2))**2  * 0.5 *rho
    return (abs(DP_f + DP_s - DP_g).magnitude)

In [10]:
V_dot_aux = 10/1000
V_dot_134_o = op.newton(flow_134_o,V_dot_aux)
V_dot_opt = V_dot_134_o*u.m**3/u.s
V_dot_opt.to(u.L/u.s)

6.553931975996452 <Unit('liter / second')>